## Add GPS Metadata capture for line and polygon layers

This notebook configures an existing line or polygon feature layer to record GPS metadata using ArcGIS Field Maps.

The notebook performs the following actions:

- Adds the esri_fixtype domain
- Adds a series of summary statistics fields
- Enables attachments if needed

Note: you must be the owner of the feature layer or an organization admin user

### Steps

1. Update the following variables
    - org_url
    - username
    - itemID
    - featureLayerIndex
2. Run all cells


In [44]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

# update this information prior to running
org_url = "https://www.arcgis.com"
username = "doug_nitro"
itemID = "7285d87a155649cf93c7826755503f0a" # itemID of feature layer
featureLayerIndex = 1 #index of layer you want to update

gis = GIS(org_url, username)

In [45]:
# get the feature layer by ID
item = gis.content.get(itemID)

# check the item type is a feature service
if (item.type != 'Feature Service'):
    raise TypeError("Item is not a feature service")

featureLayerCollection = FeatureLayerCollection.fromitem(item)

featLayer = featureLayerCollection.layers[featureLayerIndex]

# make sure layer is a line or polygon layer
if ((featLayer.properties.geometryType != 'esriGeometryPolyline') and 
    (featLayer.properties.geometryType != 'esriGeometryPolygon')):
    raise TypeError("Feature layer is not a line or polygon layer")

# check and enable attachments
if (featLayer.properties.hasAttachments == False):    
    featLayer.manager.update_definition({"hasAttachments": True})

In [46]:
# Extract existing fields from Feature layer service definition
featureLayerFields = featLayer.properties.fields

# New fields which need to be added
gnssMetadataFields = {'fields': []}

# Operations list - Add or Update GNSS Metadata fields.
operations = []

# Ensure all fields are present; if not, add them to the definition

# esrignss_avg_h_rms
avg_horizontal_accuracyField = [field for field in featureLayerFields if field['name'] == 'esrignss_avg_h_rms']

if not avg_horizontal_accuracyField:
    gnssMetadataFields['fields'].append({'name': 'esrignss_avg_h_rms',
                                            'type': 'esriFieldTypeDouble',
                                            'alias': 'Average horizontal accuracy (m)',
                                            'sqlType': 'sqlTypeOther',
                                            'nullable': True,
                                            'editable': True,
                                            'domain': None,
                                            'defaultValue': None})

# esrignss_avg_v_rms
avg_vertical_accuracyField = [field for field in featureLayerFields if field['name'] == 'esrignss_avg_v_rms']

if not avg_vertical_accuracyField:
    gnssMetadataFields['fields'].append({'name': 'esrignss_avg_v_rms',
                                            'type': 'esriFieldTypeDouble',
                                            'alias': 'Average vertical accuracy (m)',
                                            'sqlType': 'sqlTypeOther',
                                            'nullable': True,
                                            'editable': True,
                                            'domain': None,
                                            'defaultValue': None})

# esrignss_worst_h_rms
worst_horizontal_accuracyField = [field for field in featureLayerFields if field['name'] == 'esrignss_worst_h_rms']

if not worst_horizontal_accuracyField:
    gnssMetadataFields['fields'].append({'name': 'esrignss_worst_h_rms',
                                            'type': 'esriFieldTypeDouble',
                                            'alias': 'Worst horizontal accuracy (m)',
                                            'sqlType': 'sqlTypeOther',
                                            'nullable': True,
                                            'editable': True,
                                            'domain': None,
                                            'defaultValue': None})

# esrignss_worst_v_rms
worst_vertical_accuracyField = [field for field in featureLayerFields if field['name'] == 'esrignss_worst_v_rms']

if not worst_vertical_accuracyField:
    gnssMetadataFields['fields'].append({'name': 'esrignss_worst_v_rms',
                                            'type': 'esriFieldTypeDouble',
                                            'alias': 'Worst vertical accuracy (m)',
                                            'sqlType': 'sqlTypeOther',
                                            'nullable': True,
                                            'editable': True,
                                            'domain': None,
                                            'defaultValue': None})


# esrignss_worst_fixtype
worst_fixTypeField = [field for field in featureLayerFields if field['name'] == 'esrignss_worst_fixtype']

if worst_fixTypeField:
    # Field does exist check if the domain is set.
    if worst_fixTypeField[0]['domain'] == None:
        if ([operation for operation in operations if operation == 'updateDefinition']):
            operations.append('updateDefinition')

        worstfixtypeFieldIndex = featureLayerFields.index(worst_fixTypeField[0])
        fixTypeDomain = {'type': 'codedValue',
                            'name': 'ESRI_FIX_TYPE_DOMAIN',
                            'codedValues': [{'name': 'Fix not valid',
                                            'code': 0},
                                            {'name': 'GPS', 'code': 1},
                                            {'name': 'Differential GPS', 'code': 2},
                                            {'name': 'RTK Fixed', 'code': 4},
                                            {'name': 'RTK Float', 'code': 5}]}
        featureLayerFields[worstfixtypeFieldIndex]['domain'] = fixTypeDomain

else:
    gnssMetadataFields['fields'].append({'name': 'esrignss_worst_fixtype',
                                            'type': 'esriFieldTypeSmallInteger',
                                            'alias': 'Worst fix type',
                                            'sqlType': 'sqlTypeOther',
                                            'nullable': True,
                                            'editable': True,
                                            'domain': {'type': 'codedValue',
                                                    'name': 'ESRI_FIX_TYPE_DOMAIN',
                                                    'codedValues': [
                                                        {'name': 'Fix not valid', 'code': 0},
                                                        {'name': 'GPS', 'code': 1},
                                                        {'name': 'Differential GPS', 'code': 2},
                                                        {'name': 'RTK Fixed', 'code': 4},
                                                        {'name': 'RTK Float', 'code': 5}]},
                                            'defaultValue': None})

# esrignss_manual_locations
num_manual_locationsField = [field for field in featureLayerFields if
                        field['name'] == 'esriFieldTypeInteger']

if not num_manual_locationsField:
    gnssMetadataFields['fields'].append({'name': 'esrignss_manual_locations',
                                            'type': 'esriFieldTypeInteger',
                                            'alias': 'Number of manual locations',
                                            'sqlType': 'sqlTypeOther',
                                            'nullable': True,
                                            'editable': True,
                                            'domain': None,
                                            'defaultValue': None})

# Check if AddToDefinition operation needs to be added.
initialFeatureLayerFieldsCount = len(featureLayerFields)
if (len(gnssMetadataFields[
            'fields']) + initialFeatureLayerFieldsCount) > initialFeatureLayerFieldsCount:
    operations.append('addToDefinition')

In [47]:
# Update service definition
for operation in operations:

    # Add
    if operation == 'addToDefinition':
        response = featLayer.manager.add_to_definition(
            gnssMetadataFields)
        print("Successfully added GNSS fields.")

    # Modify
    else:
        response = featLayer.manager.update_definition(
            featureLayerFields)
        print("Successfully updated GNSS fields.")

    result = response['success']

    if not result:
        print('Failed to update Feature layer service definition.')
    else:
        print('Service definition updated successfully.')  

Successfully added GNSS fields.
Service definition updated successfully.
